In [5]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from PIL import Image
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
dataset_dir = '/content/drive/MyDrive/Assignment_1/dataset'
test_dir    = '/content/drive/MyDrive/Assignment_1/test'
model_path  = '/content/drive/MyDrive/Assignment_1/emoji_cnn_model_final.pth'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
img_height, img_width = 128, 128
batch_size = 16
epochs = 50
learning_rate = 1e-4
validation_split = 0.2
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
train_transforms = transforms.Compose([
    transforms.Resize((img_height, img_width)),
    transforms.RandomRotation(10),
    transforms.RandomAffine(0, translate=(0.05,0.05), shear=0.05, scale=(0.95,1.05)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.1,0.1,0.1,0.05),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])

val_transforms = transforms.Compose([
    transforms.Resize((img_height, img_width)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])

In [10]:
full_dataset = datasets.ImageFolder(dataset_dir, transform=train_transforms)
num_classes  = len(full_dataset.classes)
total_size   = len(full_dataset)
val_size     = int(validation_split * total_size)
train_size   = total_size - val_size

train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])
val_dataset.dataset.transform = val_transforms  # apply val transforms

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,  num_workers=2)
val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False, num_workers=2)

In [11]:
class EmojiCNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),  # layer 1
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1), # layer 2
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),# layer 3
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(128 * (img_height//8) * (img_width//8), 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return self.classifier(x)

model = EmojiCNN(num_classes).to(device)


In [15]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.5)

In [16]:
best_val_acc = 0.0

for epoch in range(1, epochs+1):
    # -- Train
    model.train()
    running_loss = 0.0
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * imgs.size(0)
    train_loss = running_loss / train_size


In [18]:
model.eval()
correct, total = 0, 0
val_loss = 0.0
with torch.no_grad():
    for imgs, labels in val_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        val_loss += loss.item() * imgs.size(0)
        _, preds = outputs.max(1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
val_loss /= val_size
val_acc  = correct / total

scheduler.step(val_loss)

print(f"Epoch {epoch:02d} | "
      f"Train Loss: {train_loss:.4f} | "
      f"Val Loss: {val_loss:.4f} | "
      f"Val Acc: {val_acc:.4f}")

# save best
if val_acc > best_val_acc:
    best_val_acc = val_acc
    torch.save(model.state_dict(), model_path)

print(f"Training complete! Best Validation Accuracy: {best_val_acc:.4f}")


Epoch 50 | Train Loss: 0.4163 | Val Loss: 1.5236 | Val Acc: 0.5370
Training complete! Best Validation Accuracy: 0.5370


# New Section